In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import math

import numpy as np
import pandas as pd

from backtesting import Backtest, Strategy
from backtesting.test import SMA
import tulipy as tu


Loading BokehJS ...

In [3]:
dataset_path = './data/HistoricalData_NG.csv'

In [4]:
def get_adx(data: pd.DataFrame, n: int):
    dx = tu.adx(data.High, data.Low, data.Close, n)
    return return_indicator(dx, len(data))

def get_atr(data: pd.DataFrame, n: int):
    atr = tu.atr(data.High, data.Low, data.Close, n)
    return return_indicator(atr, len(data))

def return_indicator(values, data_len: int):
    a = np.empty((data_len - len(values),))
    a[:] = np.nan
    return np.concatenate([a, values])

In [5]:
class MovingAvg3LineCross(Strategy):
    n1 = 10
    n2 = 15
    n3 = 35
    risk = 5

    def init(self):
        super(MovingAvg3LineCross, self).init()
        self.trailingATRLen = 5
        self.lookback = 5
        self.atrStopMultiplier = 1.2
        self.shortSelling = False  # allowance of short selling
        self.pointvalue = 10

        self.fastSMA = self.I(SMA, self.data.Close, self.n1)
        self.medSMA = self.I(SMA, self.data.Close, self.n2)
        self.slowSMA = self.I(SMA, self.data.Close, self.n3)
        self.atr = self.I(get_atr, self.data, 14)
        self.trailingAtr = self.I(get_atr, self.data, self.trailingATRLen)

    def next(self):
        if len(self.trades) == 0:
            if self.slowSMA[-1] < self.medSMA[-1] < self.fastSMA[-1] < self.data.Close[-1] \
                    and self.get_slope() > 0.01:
                self.buy(size=self.get_position_size())
            if self.data.Close[-1] < self.fastSMA[-1] < self.medSMA[-1] < self.slowSMA[-1] \
                    and self.shortSelling and self.get_slope() < -0.01:
                self.sell(size=self.get_position_size())
        else:
            if self.trades[0].is_long:
                if self.data.Low[-1] <= self.get_trailing_stop(long=True):
                    self.position.close()
            else:
                if self.data.High[-1] >= self.get_trailing_stop(long=False) and self.shortSelling:
                    self.position.close()

    def get_slope(self):
        y = self.data.Close[-5:]
        X = np.arange(1, len(y) + 1, 1)
        y_m, x_m = np.mean(y), np.mean(X)
        m = np.sum([(y_i - y_m)*(x_i - x_m) for y_i, x_i in zip(y, X)]) / np.sum([(x_i - x_m) ** 2 for x_i in X])
        return m

    def get_position_size(self):
        return math.floor((self.risk / 100 * self.equity) / (self.atr * self.pointvalue))

    def get_trailing_stop(self, long: bool):
        if long:
            return np.max(self.data.Low[-self.lookback:]) - self.trailingAtr[-1] * self.atrStopMultiplier
        else:
            return np.min(self.data.High[-self.lookback:]) + self.trailingAtr[-1] * self.atrStopMultiplier

In [6]:
data = pd.read_csv(dataset_path, sep=',', index_col=0, parse_dates=True)
data = data.dropna()

data = data.set_index(pd.to_datetime(data.index))
data.sort_values(by='Date', inplace=True)
data = data['2017-01-02': '2021-09-14']


bt = Backtest(data, MovingAvg3LineCross, cash=250_000, commission=0.02, trade_on_close=True, margin=0.05)

In [7]:
stats = bt.run()

stats

Start                     2018-01-02 00:00:00
End                       2021-08-30 00:00:00
Duration                   1336 days 00:00:00
Exposure Time [%]                   29.035753
Equity Final [$]                    275767.83
Equity Peak [$]                     275767.83
Return [%]                          10.307132
Buy & Hold Return [%]               40.870419
Return (Ann.) [%]                    2.714499
Volatility (Ann.) [%]                5.754934
Sharpe Ratio                         0.471682
Sortino Ratio                        0.761616
Calmar Ratio                         0.375599
Max. Drawdown [%]                   -7.227117
Avg. Drawdown [%]                   -3.189693
Max. Drawdown Duration     1017 days 00:00:00
Avg. Drawdown Duration      396 days 00:00:00
# Trades                                   14
Win Rate [%]                        35.714286
Best Trade [%]                      36.966551
Worst Trade [%]                     -15.23201
Avg. Trade [%]                    

In [8]:
bt.plot()

Row(id='1586', ...)